# **EDA y ETL: Conjunto de datos 1**

Es importante destacar que, para una persona no experta en datos interesada en conocer la situación de los delitos en su ciudad, la comprensión de este conjunto de datos puede resultar complicada, ya que se presenta como una comparación entre periodos de tiempo. Dado que nuestro objetivo es aprovechar los datos abiertos de la manera más efectiva, llevaremos a cabo una reestructuración de este conjunto de datos. Esta transformación buscará simplificar su interpretación y hacerlo más accesible, asegurando que cualquier persona pueda entender fácilmente la información presentada.

## **Librerías y modulos necesarios**

Las librerías necesarias para el desarrollo del análisis exploratorio y transformación de datos son las siguientes:

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# **Conjunto de datos 1**

Inicialmente hacemos cargue del conjunto de datos a usar. 

In [2]:
df = pd.read_csv(r"C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\comparativo_de_delitos.csv")

## **Contextualización de los datos**

Este conjunto de datos presenta un consolidado de delitos de alto impacto ocurridos en la ciudad de Barranquilla. Incluye cifras preliminares de homicidios, denuncias por extorsión, lesiones personales, delitos sexuales, violencia intrafamiliar y diversos tipos de hurto, como hurto a personas, comercios, residencias, entidades financieras, automotores y motocicletas. Además, proporciona una comparación entre el mismo periodo de diferentes años, destacando las variaciones porcentuales (%) y absolutas (#), lo que permite un análisis detallado de las tendencias y cambios en la incidencia de estos delitos a lo largo del tiempo. Este conjunto de datos constituye una herramienta valiosa para comprender la dinámica delictiva en Barranquilla y diseñar estrategias de prevención basadas en evidencia.

El conjunto de datos se encuentra en el siguiente enlace: [Comparativo de delitos de alto impacto en la ciudad de Barranquilla](https://www.datos.gov.co/Seguridad-y-Defensa/Comparativo-de-delitos-de-alto-impacto-en-la-ciuda/4p95-h82w/about_data)


### **Características de los datos**

Ahora se visualizan las primeras cinco observaciones del conjunto de datos.

In [3]:
df.head()

,Periodo meses comparado,Años comparados,Delito,Casos/denuncias anterior periodo,Casos/denuncias último periodo,Variación %,Variación absoluta,Fuente
0,01 al 31 de enero,2019-2020,Homicidios,25,23,-8%,-2,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
1,01 al 31 de enero,2019-2020,Lesiones personales o comunes,279,186,-33%,-93,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
2,01 al 31 de enero,2019-2020,Hurtos a personas,892,944,6%,52,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
3,01 al 31 de enero,2019-2020,Hurtos a comercios,171,59,-65%,-112,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
4,01 al 31 de enero,2019-2020,Hurtos a residencias,85,50,-41%,-35,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."


In [4]:
df.shape

(427, 8)

El conjunto de datos inicial cuenta con **427** observaciones y **8** variables.

## **Transformación, limpieza y reorganización del conjunto de datos**

Inicialmente, se eliminó la columna correspondiente a **"Fuente"**, ya que no será relevante para el desarrollo del tablero de visualización. No obstante, es importante mencionar que la fuente de los datos es **"POLICÍA MEBAR-GRICRI"**, y los mismos fueron extraídos del balance criminológico, garantizando la veracidad y confiabilidad de la información utilizada.

In [5]:
df.drop('Fuente', axis = 1, inplace = True)

En este proceso, se reorganizan y preparan los datos para facilitar su análisis. Primero, la columna **"Años comparados"** se divide en dos nuevas columnas, **"Año anterior"** y **"Último año"**, separando la información de los años comparados. Luego, se elimina la columna original **"Años comparados"** al no ser necesaria tras la división. Finalmente, las columnas del DataFrame se reorganizan para mejorar su legibilidad, destacando información clave como el periodo comparado, los tipos de delitos, el número de casos en cada año y las variaciones porcentuales y absolutas. Este proceso da como resultado una estructura más clara y lógica que facilita la interpretación de los datos y el análisis de tendencias delictivas.

In [6]:
# Dividir la columna 'Años comparados' en 'Año anterior' y 'Último año'
df[['Año anterior', 'Último año']] = df['Años comparados'].str.split('-', expand = True)

# Eliminar la columna 'Años comparados' original
df.drop('Años comparados', axis = 1, inplace = True)

# Reorganizar las columnas para que las nuevas columnas estén en el lugar adecuado
df = df[['Periodo meses comparado', 'Año anterior', 'Último año', 'Delito', 
         'Casos/denuncias  anterior periodo', 'Casos/denuncias último periodo', 
         'Variación %', 'Variación absoluta']]

In [7]:
df.rename(columns = {
    'Periodo meses comparado': 'Meses comparados',
    'Casos/denuncias  anterior periodo': 'Casos anterior año',
    'Casos/denuncias último periodo': 'Casos último año',
    'Variación %': 'Variacion porcentual',
    'Variación absoluta': 'Variacion absoluta'
}, inplace = True)


Para manejar la duplicidad de categorías debido a diferencias en mayúsculas y minúsculas, se aplica una transformación a la columna **"Delito"** utilizando una función lambda. Esta transformación estandariza los nombres de las categorías, agrupando aquellas que tienen el mismo significado pero están escritas de forma diferente (por ejemplo, en mayúsculas o con variaciones en su redacción). La función busca palabras clave en cada valor de la columna, las convierte a minúsculas con `.lower()` y asigna un nombre unificado a las categorías. Esto garantiza consistencia en los datos y facilita el análisis posterior al eliminar redundancias y garantizar que cada categoría represente un único tipo de delito.

In [8]:
# Aplicar la transformación usando lambda para agrupar las categorías repetidas
df['Delito'] = df['Delito'].apply(lambda x: 'Delitos sexuales' if 'delito' in str(x).lower() or 'sexuales' in str(x).lower() else (
    'Hurtos a comercios' if 'hurto a comer' in str(x).lower() else (
    'Hurtos a personas' if 'hurto a persona' in str(x).lower() or 'hurto a personas' in str(x).lower() else (
    'Hurtos de motocicletas' if 'hurto motocicletas' in str(x).lower() or 'hurto a motocicletas' in str(x).lower() or 'hurtos motocicletas' in str(x).lower() else (
    'Hurtos a residencias' if 'hurto a residencias' in str(x).lower() else (
    'Hurtos automotores' if 'hurto automotores' in str(x).lower() or 'hurto a automotores' in str(x).lower() else (
    'Lesiones personales o comunes' if 'lesiones personales' in str(x).lower() else (
    'Extorsión' if 'extorsión' in str(x).lower() else (
    'Violencia intrafamiliar' if 'violencia intrafamiliar' in str(x).lower() else (
    'Hurtos a entidades financieras' if 'hurto a entidades financieras' in str(x).lower() or 'hurto a entidad financiera' in str(x).lower() else (
    'Hurtos a residencias' if 'hurto a residencia' in str(x).lower() else (
    'Hurtos de bicicletas' if 'hurto a bicicletas' in str(x).lower() else (
    'Hurtos de celulares' if 'hurto a celulares' in str(x).lower() else x)))))))))))))

Entonces, los delitos que encontramos en la base de datos son los siguentes: 

In [9]:
df.value_counts('Delito')

Delito
Homicidios                        47
Hurtos a comercios                47
Hurtos a personas                 47
Hurtos a residencias              47
Hurtos automotores                47
Hurtos de motocicletas            47
Lesiones personales o comunes     47
Hurtos a entidades financieras    32
Extorsión                         26
Delitos sexuales                  15
Violencia intrafamiliar           15
Hurtos de bicicletas               5
Hurtos de celulares                5
Name: count, dtype: int64

Ahora, haremos una transformación por cada delito.

En este bloque de código se realiza una transformación específica para el delito de **homicidios**, pero este proceso es aplicable a todos los delitos mencionados previamente (delitos sexuales, extorsión, violencia intrafamiliar, hurtos a personas, comercios, residencias, entidades financieras, automotores, motocicletas, bicicletas y celulares). A continuación, se describe lo que se realiza:

1. **Filtrado por delito específico**:
   - Se seleccionan únicamente las filas correspondientes al delito de interés, en este caso, **homicidios**.

2. **Extracción y limpieza de datos**:
   - Se extrae el nombre del mes desde la columna **"Meses comparados"** para obtener una columna separada con el nombre del mes.
   - La primera letra de cada mes se convierte en mayúscula para estandarizar el formato.

3. **Conversión de columnas numéricas**:
   - Se convierten las columnas **"Casos anterior año"** y **"Casos último año"** a valores numéricos para permitir cálculos matemáticos posteriores.

4. **Cálculo de casos mensuales**:
   - Se crean nuevas columnas que calculan la diferencia de los casos de cada mes dentro de los años respectivos, asegurando que se puedan comparar de manera precisa.

5. **Cálculo de variaciones**:
   - Se calcula la variación absoluta entre los casos del año anterior y los del último año.
   - Se calcula la variación porcentual, mostrando cómo ha cambiado el número de casos entre ambos años.

6. **Renombrado y reorganización de columnas**:
   - Las columnas se renombran para que sean más descriptivas.
   - Se reorganizan para mejorar la legibilidad y facilitar la exportación.

7. **Conversión a enteros y exportación**:
   - Las columnas numéricas se convierten a enteros eliminando los decimales, y los resultados se exportan a un archivo CSV con los datos ya ordenados y procesados.

Este proceso permite organizar y analizar los datos de manera detallada para cada delito, estandarizando los cálculos y dejando los datos listos para su interpretación y visualización en herramientas como Power BI.

In [10]:
df_homicidios = df[df['Delito'] == 'Homicidios']
# Extraer solo el mes de "Meses comparados" tomando la última palabra
df_homicidios['Mes'] = df_homicidios['Meses comparados'].str.split().str[-1]

# Convertir la primera letra de cada mes a mayúscula
df_homicidios['Mes'] = df_homicidios['Mes'].str.capitalize()

# Convertir las columnas de "Casos" a valores numéricos
df_homicidios['Casos anterior año'] = pd.to_numeric(df_homicidios['Casos anterior año'], errors='coerce')
df_homicidios['Casos último año'] = pd.to_numeric(df_homicidios['Casos último año'], errors='coerce')

# Crear una nueva columna con la diferencia de los casos mensuales para el año anterior
df_homicidios['Casos mensuales anterior año'] = df_homicidios['Casos anterior año'].diff().fillna(df_homicidios['Casos anterior año'])

# Crear una nueva columna con la diferencia de los casos mensuales para el último año
df_homicidios['Casos mensuales último año'] = df_homicidios['Casos último año'].diff().fillna(df_homicidios['Casos último año'])

# Agrupar por "Año anterior" para asegurar que la diferencia se calcule correctamente dentro de cada año
df_homicidios['Casos mensuales anterior año'] = df_homicidios.groupby('Año anterior')['Casos anterior año'].diff().fillna(df_homicidios['Casos anterior año'])
df_homicidios['Casos mensuales último año'] = df_homicidios.groupby('Año anterior')['Casos último año'].diff().fillna(df_homicidios['Casos último año'])

# Eliminar las columnas que no son necesarias para el análisis
df_homicidios.drop(columns=['Meses comparados', 'Casos anterior año', 'Casos último año'], inplace = True)

# Renombrar las columnas para que sean más descriptivas
df_homicidios.rename(columns={
    'Casos mensuales anterior año': 'Casos Mensuales Año Anterior',
    'Casos mensuales último año': 'Casos Mensuales Último Año'
}, inplace=True)
# Calcular la variación absoluta
df_homicidios['Variacion absoluta'] = df_homicidios['Casos Mensuales Último Año'] - df_homicidios['Casos Mensuales Año Anterior']

# Calcular la variación porcentual
df_homicidios['Variacion porcentual'] = ((df_homicidios['Casos Mensuales Último Año'] - df_homicidios['Casos Mensuales Año Anterior']) / df_homicidios['Casos Mensuales Año Anterior']) * 100
df_homicidios['Variacion porcentual'] = df_homicidios['Variacion porcentual'].round(1)

# Reubicar las columnas en el orden que prefieras
df_homicidios = df_homicidios[['Mes', 'Año anterior', 'Último año', 'Delito', 'Casos Mensuales Año Anterior', 'Casos Mensuales Último Año', 'Variacion porcentual', 'Variacion absoluta']]

# Seleccionamos las columnas numéricas (excepto las categóricas y las de los años)
numerical_columns = df_homicidios.select_dtypes(include=['float64']).columns

# Convertimos esas columnas a enteros (eliminando los decimales)
df_homicidios[numerical_columns] = df_homicidios[numerical_columns].astype(int)

df_homicidios.to_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\homicidios_ordenados.csv', index=False)

Cabe destacar que, en este caso, el proceso se muestra únicamente para la categoría de **homicidios** con el objetivo de mejorar la claridad y visibilidad del informe. No obstante, este mismo procedimiento fue aplicado de manera consistente a cada uno de los tipos de delitos mencionados anteriormente, garantizando un análisis detallado y uniforme para todas las categorías incluidas en el conjunto de datos.

Ahora, importamos cada base de datos generada por la transformación. 

In [11]:
df_homicidios = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\homicidios_ordenados.csv")
df_hurtos_comercios = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_comercios_ordenados.csv")
df_hurtos_personas = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_personas_ordenados.csv")
df_hurtos_residencias = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_residencias_ordenados.csv")
df_hurtos_automotores = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_automotores_ordenados.csv")
df_hurtos_motocicletas = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_motocicletas_ordenados.csv")
df_lesiones_personales = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\lesiones_personales_ordenados.csv")
df_extorsion = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\extorsion_ordenados.csv")
df_delitos_sexuales = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\delitos_sexuales_ordenados.csv")
df_violencia_intrafamiliar  = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\violencia_intrafamiliar_ordenados.csv")
df_hurtos_bicicletas = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_bicicletas_ordenados.csv")
df_hurtos_celulares = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_celulares_ordenados.csv")

Se unen cada base de datos de tipo de delitos en una sola llamada `df_total`

In [12]:
# Concatenar todos los DataFrames en uno solo
df_total = pd.concat([
    df_homicidios,
    df_hurtos_comercios,
    df_hurtos_personas,
    df_hurtos_residencias,
    df_hurtos_automotores,
    df_hurtos_motocicletas,
    df_lesiones_personales,
    df_extorsion,
    df_delitos_sexuales,
    df_violencia_intrafamiliar,
    df_hurtos_bicicletas,
    df_hurtos_celulares
], ignore_index=True)

En este bloque de código se limpia y ajusta el conjunto de datos consolidado. Primero, se eliminan todas las filas que contienen valores faltantes (*NaN*) utilizando `dropna()`, asegurando que los datos estén completos. Luego, se identifican las columnas numéricas de tipo `float64` con `select_dtypes` y se convierten a enteros (`int`) para eliminar los decimales, lo cual es útil en análisis donde no se necesitan valores fraccionarios, como el conteo de casos. Finalmente, con `df_total.shape`, se verifica que el DataFrame resultante contiene **330 filas** y **8 columnas**, indicando que los datos están limpios y listos para su análisis.

In [13]:
# Eliminar filas con valores NaN
df_total = df_total.dropna()

# Seleccionar las columnas numéricas
numerical_columns = df_total.select_dtypes(include=['float64']).columns

# Convertir las columnas numéricas a enteros, eliminando los decimales
df_total[numerical_columns] = df_total[numerical_columns].astype(int)

In [14]:
df_total.shape

(330, 8)

Dado que, según la estructura de los datos, los casos del año **2023** se encuentran en la columna **"Último año"**, mientras que los casos de años anteriores están en la columna **"Año anterior"**, se procede a separar y filtrar la información. Primero, se crea un DataFrame filtrado (`df_filtered`) con las columnas relevantes para los casos de años anteriores. Luego, se extraen únicamente los registros correspondientes al año **2023** en un nuevo DataFrame (`df_2023`). Este último conserva las columnas específicas de **mes**, **año**, **delito**, y los **casos mensuales del último año**, lo que permite analizar los casos del año más reciente de manera independiente. Esto asegura que la información esté correctamente segmentada para facilitar su análisis y comparación.

In [15]:
df_filtered = df_total[['Mes', 'Año anterior', 'Delito', 'Casos Mensuales Año Anterior']]
df_2023 = df_total[df_total['Último año'] == 2023]
df_2023 = df_2023[['Mes', 'Último año','Delito', 'Casos Mensuales Último Año']]

In [16]:
df_2023.rename(columns={
    'Mes': 'Mes',
    'Último año': 'Año anterior',
    'Delito': 'Delito',
    'Casos Mensuales Último Año': 'Casos Mensuales Año Anterior'
}, inplace=True)

En este bloque de código, se integran los datos del año **2023** con los de años anteriores para crear un DataFrame consolidado. Primero, las columnas de `df_2023` se renombran para que coincidan con la estructura de `df_filtered`, lo que asegura consistencia en los nombres de las columnas. Luego, ambos DataFrames se concatenan utilizando `pd.concat`, generando un único conjunto de datos (`df_final`) con todos los años de análisis. Finalmente, se ajustan los valores en la columna **"Casos Mensuales Año Anterior"**, reemplazando posibles valores negativos por 0 mediante una función `lambda`, garantizando datos coherentes y listos para su análisis.

In [17]:
df_final = pd.concat([df_filtered, df_2023], ignore_index = True)

In [18]:
df_final['Casos Mensuales Año Anterior'] = df_final['Casos Mensuales Año Anterior'].apply(lambda x: x if x >= 0 else 0)

En este paso, se renombran las columnas del DataFrame consolidado (`df_final`) para hacerlas más claras y descriptivas. Por ejemplo, **"Año anterior"** se cambia a **"Año"** y **"Casos Mensuales Año Anterior"** se convierte en **"Cantidad de casos"**, lo que facilita la interpretación de los datos. Este cambio asegura que los nombres de las columnas sean más intuitivos y adecuados para su análisis y visualización, mejorando la legibilidad del conjunto de datos.

In [19]:
df_final.rename(columns={
    'Mes': 'Mes',
    'Año anterior': 'Año',
    'Delito': 'Delito',
    'Casos Mensuales Año Anterior': 'Cantidad de casos'
}, inplace=True)


Este proceso transforma el conjunto de datos para que sea compatible y óptimo para su uso en **Power BI**, asegurando un manejo eficiente y visualmente claro. Se realiza un mapeo de los meses a valores numéricos y se crea una columna de **"Fecha"** combinando el año, el mes y un día estándar ("01"), permitiendo trabajar con fechas en un formato adecuado para gráficos de series temporales. Además, se eliminan columnas intermedias y originales como **"Mes"** y **"Año"**, se reorganizan las columnas para priorizar **"Fecha"**, **"Delito"** y **"Cantidad de casos"**, y se reemplazan valores negativos en los datos por 0. Finalmente, el conjunto de datos procesado se exporta a un archivo CSV, listo para ser utilizado en tableros de visualización en Power BI.

In [20]:
# Crear un mapeo para los meses
meses = {
    "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4, "Mayo": 5,
    "Junio": 6, "Julio": 7, "Agosto": 8, "Septiembre": 9, "Octubre": 10,
    "Noviembre": 11, "Diciembre": 12
}

# Convertir el mes a número usando el mapeo
df_final['Mes_numero'] = df_final['Mes'].map(meses)

# Crear una nueva columna de fecha con el formato Año-Mes
df_final['Fecha'] = pd.to_datetime(df_final['Año'].astype(str) + '-' + df_final['Mes_numero'].astype(str) + '-01')

# Ahora puedes eliminar las columnas adicionales si ya no las necesitas
df_final.drop(columns=['Mes_numero'], inplace=True)

# Reemplazar los valores negativos en 'Cantidad de casos' por 0 (si es necesario)
df_final['Cantidad de casos'] = df_final['Cantidad de casos'].apply(lambda x: max(0, x))

# Crear la columna 'Fecha' con solo el año y el mes
df_final['Fecha'] = df_final['Fecha'].dt.to_period('M')

# Eliminar las columnas originales 'Mes' y 'Año'
df_final.drop(columns=['Mes', 'Año'], inplace=True)

# Reorganizar las columnas, poniendo 'Fecha' al principio
df_final = df_final[['Fecha', 'Delito', 'Cantidad de casos']]

# Exportar el DataFrame df_final a la ruta especificada
df_final.to_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\df_final.csv', index=False)